In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import mysql.connector
from mysql.connector import Error
from datetime import date


app = Flask(__name__)

def get_db_connection():
    conn = mysql.connector.connect(
        host="localhost",
        user="admin",
        password="helloLayka",
        database="test"
    )
    return conn



@app.route('/configuration')
def configuration():
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)

    # Fetching all rows for OWNER, CATEGORY, DOC_STATE, REVIEW_STATE
    cursor.execute('SELECT * FROM OWNER')
    owner_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM CATEGORY')
    category_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM DOC_STATE')
    doc_state_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM REVIEW_STATE')
    review_state_rows = cursor.fetchall()

    # Query to join DOCUMENT with OWNER and CATEGORY tables
    cursor.execute('''
        SELECT DOCUMENT.id, DOCUMENT.name, DOCUMENT.initials, OWNER.name AS owner_name, CATEGORY.name AS category_name, DOCUMENT.owner_id, DOCUMENT.category_id
        FROM DOCUMENT
        JOIN OWNER ON DOCUMENT.owner_id = OWNER.id
        JOIN CATEGORY ON DOCUMENT.category_id = CATEGORY.id
    ''')
    document_rows = cursor.fetchall()

    # Fetching lists for dropdowns in the DOCUMENT table
    cursor.execute('SELECT id, name FROM OWNER')
    owner_list = cursor.fetchall()

    cursor.execute('SELECT id, name FROM CATEGORY')
    category_list = cursor.fetchall()

    cursor.close()
    conn.close()

    return render_template('configuration.html',
                           owner_rows=owner_rows, 
                           category_rows=category_rows, 
                           doc_state_rows=doc_state_rows, 
                           review_state_rows=review_state_rows,
                           document_rows=document_rows,
                           owner_list=owner_list,
                           category_list=category_list)

@app.route('/modify/<table_name>/<int:row_id>', methods=['POST'])
def modify_row(table_name, row_id):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Build the update query dynamically based on the form fields
    update_fields = ', '.join([f"{key} = %s" for key in request.form.keys()])
    values = list(request.form.values())

    cursor.execute(f"UPDATE {table_name} SET {update_fields} WHERE id = %s", values + [row_id])
    conn.commit()
    cursor.close()
    conn.close()

    return redirect(url_for('configuration'))



@app.route('/delete/<table_name>/<int:row_id>', methods=['POST'])
def delete_row(table_name, row_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute(f'DELETE FROM {table_name} WHERE id = %s', (row_id,))
        conn.commit()
    except Error as e:
        # Check if the error is due to a foreign key constraint
        if e.errno == 1451:  # MySQL error code for foreign key constraint failure
            cursor.close()
            conn.close()
            return f"Cannot delete this item because it is still in use. The {table_name} entry with ID {row_id} could not be deleted because it is referenced by another table.", 400
        else:
            raise e  # Re-raise if it's another kind of error
    
    cursor.close()
    conn.close()
    
    return redirect(url_for('configuration'))

@app.route('/add_row/<table_name>', methods=['POST'])
def add_row(table_name):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Extract the form data, excluding the id field since it's usually auto-incremented
    keys = ', '.join(request.form.keys())
    placeholders = ', '.join(['%s'] * len(request.form))
    values = list(request.form.values())

    cursor.execute(f"INSERT INTO {table_name} ({keys}) VALUES ({placeholders})", values)
    conn.commit()
    cursor.close()
    conn.close()

    return redirect(url_for('configuration'))


@app.route('/display_documents')
def display_documents():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Fetch documents from the DOCUMENT table
        cursor.execute("SELECT name, category_id FROM DOCUMENT ORDER BY category_id")
        documents = cursor.fetchall()
        
        # Fetch categories from the CATEGORY table
        cursor.execute("SELECT id, name FROM CATEGORY")
        categories = cursor.fetchall()
        
        # Create a dictionary to store documents by category
        documents_by_category = {}
        for document in documents:
            if document[1] not in documents_by_category:
                documents_by_category[document[1]] = []
            documents_by_category[document[1]].append(document[0])
        
        # Create a dictionary to store category names
        category_names = {category[0]: category[1] for category in categories}
        
        # Calculate the maximum number of documents in any category
        max_docs = max(len(docs) for docs in documents_by_category.values())
        
        # Return the template with the documents, categories, and max_docs
        return render_template('display_documents.html', 
                               documents_by_category=documents_by_category, 
                               category_names=category_names, 
                               max_docs=max_docs)
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    finally:
        cursor.close()
        conn.close()





@app.route('/add_specification', methods=['GET', 'POST'])
def add_specification():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    if request.method == 'POST':

        # Fetch data from the form
        title = request.form['title']
        owner_id = request.form['owner_id']
        document_id = request.form['document_id']
        
        # Generate the current date
        current_date = date.today().strftime("%Y-%m-%d")
        
        # Insert data into the SPECIFICATION table
        query = ("INSERT INTO SPECIFICATION "
                 "(title, owner_id, document_id, date) "
                 "VALUES (%s, %s, %s, %s)")
        cursor.execute(query, (title, owner_id, document_id, current_date))
        conn.commit()
        
        return redirect(url_for('success'))
    
    # Fetch owners from the OWNER table for the dropdown list
    cursor.execute("SELECT id, name FROM OWNER")
    owners = cursor.fetchall()
    # Fetch documents from the DOCUMENT table for the dropdown list
    cursor.execute("SELECT id, name FROM DOCUMENT")
    documents = cursor.fetchall()

    
    return render_template('add_specification.html', owners=owners, documents=documents)

@app.route('/success')
def success():
    return 'Specification added successfully!'




if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.30:5001
Press CTRL+C to quit
192.168.1.18 - - [02/Sep/2024 21:49:49] "GET /display_documents HTTP/1.1" 200 -
192.168.1.18 - - [02/Sep/2024 21:50:30] "GET /configuration HTTP/1.1" 200 -
192.168.1.18 - - [02/Sep/2024 21:51:12] "POST /add_row/DOCUMENT HTTP/1.1" 302 -
192.168.1.18 - - [02/Sep/2024 21:51:12] "GET /configuration HTTP/1.1" 200 -
